In [1]:
from PIL import Image
import os.path
import numpy as np
import itertools

In [116]:
def resizeImg(action, x , y):
    avgX = x
    avgY = y
    folder = ['PNEUMONIA','NORMAL']
    for foldername in folder:
        filelist= [file for file in os.listdir('chest_xray/' + action + '/' + foldername + '/') if file.endswith('.jpeg')]
        for filename in filelist:
            img = Image.open('chest_xray/' + action + '/' + foldername + '/' + filename)
            imgResize = img.resize((avgX, avgY), Image.LANCZOS)
            imgResize.save('chest_xray/' + action + '/resized250x250/' + foldername + '/' + filename)
            img.close()

In [117]:
resizeImg('test',250,250)

In [ ]:
resizeImg('train',250,250)

In [6]:
def convertImg(action):
    #convert all RGB files to grey scale. RGB is 3d and greyscale is 2d
    folder = ['PNEUMONIA','NORMAL']
    for foldername in folder:
        filelist= [file for file in os.listdir('chest_xray/' + action + '/resized250x250/' + foldername + '/') if file.endswith('.jpeg')]
        for filename in filelist:
        #for filename in itertools.islice(filelist, 0, 50):
            img = Image.open('chest_xray/' + action + '/resized250x250/' + foldername + '/' + filename)
            imgData = list(img.getdata())
            imgData = np.asarray(imgData)
            #print(imgData.shape)
            #print(imgData.ndim)
            #x = imgData.reshape(150,150)
            #The array is 1d before reshaping for grey scale images and 2d for RGB. We find all the RGB images and
            #convert them to grey scale and save with prefix 'gs_'
            if imgData.ndim == 2:
                img_gs = Image.open('chest_xray/' + action + '/resized250x250/' + foldername + '/' + filename).convert('L')
                img_gs.save('chest_xray/' + action + '/resized250x250/' + foldername + '/gs_'+ filename)
                img_gsData = list(img_gs.getdata())
                img_gsData = np.asarray(img_gsData)
                #print(filename)
                #print(img_gsData.ndim)

In [ ]:
convertImg('train')

In [ ]:
convertImg('test')

In [145]:
def process_image(action):
    X = []
    y = []
    folder = ['PNEUMONIA','NORMAL']

    for foldername in folder:
        filelist= [file for file in os.listdir('chest_xray/' + action + '/resized250x250/' + foldername + '/') if file.endswith('.jpeg')]
        for filename in filelist:
        #for filename in itertools.islice(filelist, 0, 5):
            #print(filename)
            img = Image.open('chest_xray/' + action + '/resized250x250/' + foldername + '/' + filename)
            #imgData = list(img.getdata())
            #imgData = np.asarray(imgData)
            #imgData = img.getdata()
            #imgData = np.asarray(imgData)
            rgbimg = Image.new("RGB", img.size)
            rgbimg.paste(img)
            imgData = np.asarray(rgbimg)
            #imgData = np.asarray(img)
            #print(imgData.shape)
            #print(imgData.ndim)
            if imgData.ndim == 3:
                x = imgData.reshape(250,250,3)
                X.append(x)
                if foldername == 'NORMAL':
                    y.append(0)
                else:
                    y.append(1)
            img.close()
    return X, y

In [146]:
X_train, y_train = process_image('train')

In [147]:
X_train = np.asarray(X_train)
y_train = np.asarray(y_train)
print("shape of X_train: " + str(X_train.shape))
print("length of X_train: " + str(len(X_train)))
print("shape of y_train: " + str(y_train.shape))
print("length of y_train: " + str(len(y_train)))
print("value of y_train[0]: " + str(y_train[0]))

shape of X_train: (5499, 250, 250, 3)
length of X_train: 5499
shape of y_train: (5499,)
length of y_train: 5499
value of y_train[0]: 1


In [149]:
img = Image.fromarray(X_train[1], 'RGB')
img.show()

In [150]:
y_train[1]

1

In [151]:
X_test, y_test = process_image('test')

In [152]:
X_test = np.asarray(X_test)
y_test = np.asarray(y_test)
print("shape of X_test: " + str(X_test.shape))
print("length of X_test: " + str(len(X_test)))
print("shape of y_test: " + str(y_test.shape))
print("length of y_test: " + str(len(y_test)))
print("value of y_test[0]: " + str(y_test[0]))

shape of X_test: (624, 250, 250, 3)
length of X_test: 624
shape of y_test: (624,)
length of y_test: 624
value of y_test[0]: 1


In [153]:
img = Image.fromarray(X_test[1], 'RGB')
img.show()

In [154]:
y_test[1]

1

In [155]:
from keras.utils.np_utils import to_categorical

y_train = to_categorical(y_train, 2)
y_test = to_categorical(y_test, 2)

In [156]:
from keras.callbacks import ReduceLROnPlateau , ModelCheckpoint , LearningRateScheduler
lr_reduce = ReduceLROnPlateau(monitor='val_acc', factor=0.1, epsilon=0.0001, patience=1, verbose=1)

In [157]:
filepath="transferlearning_weights.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

In [158]:
from keras.models import Sequential , Model
from keras.layers import Dense , Activation
from keras.layers import Dropout , GlobalAveragePooling2D
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD , RMSprop , Adadelta , Adam
from keras.layers import Conv2D , BatchNormalization
from keras.layers import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
K.set_image_dim_ordering('th')
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

In [161]:
X_train=X_train.reshape(5499,3,250,250)
X_test=X_test.reshape(624,3,250,250)

In [160]:
from keras.applications.inception_v3 import InceptionV3
# create the base pre-trained model
base_model = InceptionV3(weights=None, include_top=False , input_shape=(3, 250, 250))

In [162]:
x = base_model.output
x = Dropout(0.5)(x)
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = BatchNormalization()(x)
predictions = Dense(2, activation='sigmoid')(x)

In [163]:
model = Model(inputs=base_model.input, outputs=predictions)

In [164]:
model.compile(loss='categorical_crossentropy', 
                  optimizer='adam', 
                  metrics=['accuracy'])

In [165]:
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 3, 250, 250)  0                                            
__________________________________________________________________________________________________
conv2d_95 (Conv2D)              (None, 32, 124, 124) 864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_96 (BatchNo (None, 32, 124, 124) 96          conv2d_95[0][0]                  
__________________________________________________________________________________________________
activation_95 (Activation)      (None, 32, 124, 124) 0           batch_normalization_96[0][0]     
__________________________________________________________________________________________________
conv2d_96 

In [166]:
batch_size = 64
epochs = 10

In [ ]:
history = model.fit(X_train, y_train, validation_data = (X_test , y_test),callbacks=[lr_reduce,checkpoint] ,
          epochs=epochs)

Train on 5499 samples, validate on 624 samples
Epoch 1/10
